In [2]:
#!pip install pandas

In [2]:
import json
import pandas as pd
from collections import defaultdict
from itertools import combinations,permutations 
import numpy as np
from sklearn import cluster
import glob,os
import pickle
import warnings
warnings.filterwarnings("ignore")

#### Data Processing

In [4]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/"
all_datasets = glob.glob(root + 'split_merged_dataset/*_train.jsonl')
all_datasets
all_datasets[3].split('/')[-1][:-6].split('_')[0]

'ta'

In [5]:
languages = []
for dataset in all_datasets:
    lang = dataset.split('/')[-1][:-6].split('_')[0]
    languages.append(lang)
print(languages)

['pa', 'gu', 'hi', 'ta', 'as', 'ml', 'te', 'kn', 'bn', 'or', 'mr', 'en']


In [6]:
print(all_datasets)

['/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/pa_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/gu_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/hi_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/ta_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/as_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/ml_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/te_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/kn_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/bn_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/or_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/mr_merged_train.jsonl'

In [63]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/"
ent_dict = defaultdict()
fact_dict_all = defaultdict()

languages = []

for subdir, dirs, files in os.walk(root):
    print(subdir)
    
    lang =  subdir.split('/')[-1]
    print(lang)
    
    print(files)
    
    e_dict = {}
    #i=0
    if lang != "":
        with open(subdir+'/train.jsonl') as f:
            unique_facts = []

            for line in f:
#                 if i>10:
#                     break
#                 #i+=1
#                 print(i)
            
                data = json.loads(line)
                f = []
                for l in data['facts_list']:
                    #print('sentence')
                    f2 = []
                    for fact in l:
                        f2.append(fact[0])  
                        # Generate facts 
                        unique_facts.append(fact[0])  
                    f.append(f2)
                    #print(f)
                e_dict[data['entity_name']] = f
                i+=1
        languages.append(lang)
        fact_dict_all[lang] = set(unique_facts)
        print(lang,len(fact_dict_all[lang]))
        ent_dict[lang] = e_dict

/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/

['add_lang.py']
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/mr
mr
['train.jsonl', 'test.jsonl', 'val.jsonl']
mr 172
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/gu
gu
['train.jsonl', 'test.jsonl', 'val.jsonl']
gu 122
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/te
te
['train.jsonl', 'test.jsonl', 'val.jsonl']
te 151
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/pa
pa
['train.jsonl', 'test.jsonl', 'val.jsonl']
pa 201
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/ml
ml
['train.jsonl', 'test.jsonl', 'val.jsonl']
ml 223
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/kn
kn
['train.jsonl', 'test.jsonl', 'val.jsonl']
kn 150
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/bn
bn
['train.jsonl', 'test.jsonl', 'val.jsonl']
bn 234
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/hi
hi
['train.jsonl', 'test.jsonl', 'val.jsonl']
hi 207
/Users/rahulmehta/Desktop/Mult

In [24]:
#languages

In [25]:
# ent_dict = defaultdict()
# fact_dict_all = defaultdict()

# languages = []
# for dataset in all_datasets:
#     lang = dataset.split('/')[-1][:-6].split('_')[0]
    
#     e_dict = {}
#     print(lang)
#     with open(dataset) as f:
        
        
#         unique_facts = []

#         for line in f:
#     #         if i>50:
#     #             break
#             #print(i)
#             data = json.loads(line)
#             f = []
#             for l in data['facts_list']:
#                 #print('sentence')
#                 f2 = []
#                 for fact in l:
#                     f2.append(fact[0])  
#                     # Generate facts 
#                     unique_facts.append(fact[0])  
#                 f.append(f2)
#                 #print(f)
#             e_dict[data['entity_name']] = f
#             i+=1
#     languages.append(lang)
#     fact_dict_all[lang] = set(unique_facts)
#     print(lang,len(fact_dict_all[lang]))
#     ent_dict[lang] = e_dict

#### Affinity Matrix

In [505]:
#print(languages)

In [64]:
fact_dict = defaultdict()
factsetdict = defaultdict()

for lang in languages:
    print(lang)
    fdict = {i:f for i,f in enumerate(fact_dict_all[lang])}
    
    factsetdict[lang] = set(fact_dict_all[lang])
    
    fact_dict[lang] = fdict
    #print(fact_dict) 

mr
gu
te
pa
ml
kn
bn
hi
as
en
or
ta


In [65]:
perm_dict = defaultdict()
for i,(k,facts) in enumerate(factsetdict.items()):
    perm_dict[k] = list(permutations(facts,2))

#### PMI 

In [14]:
# data=pd.read_csv('../datasets/fact_pair_probs/en.tsv',sep='\t',header=None)
# data.columns = ['relation','scores']
# data.head(5)

In [66]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/"
all_datasets = glob.glob(root + 'fact_pair_probs/*.tsv')
all_datasets 

['/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/ta.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/bn.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/mr.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/te.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/as.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/gu.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/hi.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/en.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/pa.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/kn.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/or.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/ml.tsv']

In [35]:
languages

['mr', 'gu', 'te', 'pa', 'ml', 'kn', 'bn', 'hi', 'as', 'en', 'or', 'ta']

In [67]:
pmi_dict = defaultdict()
for lang in languages:
    for dataset in all_datasets:
        #print(dataset.split('/')[-1].split('.')[0])
        if lang == dataset.split('/')[-1].split('.')[0]:
            print(lang)
            pmi_list = []
            with open(dataset) as f:
                for line in f:
                    l = line.split('\t')
                    pmi_list.append(l)
    pmi_dict[lang] = pmi_list

mr
gu
te
pa
ml
kn
bn
hi
as
en
or
ta


In [47]:
#pmi_dict

#### Affinity Matrix

In [68]:
affinity_dict = defaultdict()
for lang in languages: 
    print(lang)
    a_dict = defaultdict()
    for p in perm_dict[lang]:
        #print(p)
        look = p[0] + '|' +  p[1]
        look2 = p[1] + '|' +  p[0]
        for item in pmi_dict[lang]:
            if look == item[0] or look2 == item[0]:
                # print("yes")
                #print(p)
                a_dict[p] = float(item[1].split('\n')[:-1][0])
                break
            else:
                a_dict[p] = float(0)
    affinity_dict[lang] = a_dict

mr
gu
te
pa
ml
kn
bn
hi
as
en
or
ta


In [96]:
affinity_dict

defaultdict(None,
            {'mr': defaultdict(None,
                         {('distributed by',
                           'languages spoken, written or signed'): 0.0,
                          ('distributed by', 'nominated for'): 0.0,
                          ('distributed by', 'conflict'): 0.0,
                          ('distributed by', 'point in time'): 0.0,
                          ('distributed by', 'architect'): 0.0,
                          ('distributed by', 'film editor'): 0.0,
                          ('distributed by', 'country for sport'): 0.0,
                          ('distributed by', 'award received'): 0.0,
                          ('distributed by', 'Roman praenomen'): 0.0,
                          ('distributed by', 'academic degree'): 0.0,
                          ('distributed by', 'religious name'): 0.0,
                          ('distributed by', 'work location'): 0.0,
                          ('distributed by', 'child'): 0.0,
                     

#### Create affinity matrix

In [98]:
for lang in languages:
    print(lang)
    #if lang=='en':
    s = len(fact_dict[lang]),len(fact_dict[lang])
    affinity_matrix  = np.zeros(s,dtype=float)
    #print(s)
    for r in fact_dict[lang].keys():
        #print(r)
        cols = np.zeros(len(fact_dict[lang].keys()))
        for c in fact_dict[lang].keys():
            for item in affinity_dict[lang].keys():
               # print(item)
                #print(r,item[0])
                if fact_dict[lang][r] == item[0] and fact_dict[lang][c] == item[1]:
                    #print("yes")
                    #print(affinity_dict[lang][item])
                    affinity_matrix[r][c] = affinity_dict[lang][item]
                    affinity_matrix[c][r] = affinity_dict[lang][item]
    np.save(f'../datasets/affinity_matrices/{lang}_affinity_matrix.npy',affinity_matrix)   
    print(affinity_matrix.shape)

mr
(172, 172)
gu
(122, 122)
te
(151, 151)
pa
(201, 201)
ml
(223, 223)
kn
(150, 150)
bn
(234, 234)
hi
(207, 207)
as
(132, 132)
en
(293, 293)
or
(137, 137)
ta
(212, 212)


##### Spectral clustering for all entities on validation set

In [160]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/"
all_datasets = glob.glob(root  + '*_test.jsonl')
# all_datasets 

['/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/te_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/gu_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/as_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/or_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/ta_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/bn_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/hi_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/pa_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/en_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/mr_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/ml_merged_test.jsonl',

In [71]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/"
ent_dict = defaultdict()
cnts_dict = {}
languages = []


for subdir, dirs, files in os.walk(root):
    print(subdir)
    
    lang =  subdir.split('/')[-1]
    print(lang)
    
    print(files)
    
    e_dict = {}
    cnts = []
    
    if lang != "":
        with open(subdir+'/test.jsonl') as f:
            unique_facts = []

            for line in f:
                c = 0
                data = json.loads(line)
                f = []
                for l in data['facts_list']:
                    #print('sentence')
                    f2 = []
                    for fact in l:
                        f2.append(fact[0])  
                        # Generate facts 
                        unique_facts.append(fact[0])  
                    f.append(f2)
                    #print(f)
                    c+=1

                cnts.append(c)

                e_dict[data['entity_name']] = f

        languages.append(lang)
        #fact_dict_all[lang] = set(unique_facts)
        print(lang,len(fact_dict_all[lang]))
        ent_dict[lang] = e_dict
        cnts_dict[lang] = cnts

/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/

['add_lang.py']
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/mr
mr
['train.jsonl', 'test.jsonl', 'val.jsonl']
mr 172
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/gu
gu
['train.jsonl', 'test.jsonl', 'val.jsonl']
gu 122
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/te
te
['train.jsonl', 'test.jsonl', 'val.jsonl']
te 151
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/pa
pa
['train.jsonl', 'test.jsonl', 'val.jsonl']
pa 201
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/ml
ml
['train.jsonl', 'test.jsonl', 'val.jsonl']
ml 223
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/kn
kn
['train.jsonl', 'test.jsonl', 'val.jsonl']
kn 150
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/bn
bn
['train.jsonl', 'test.jsonl', 'val.jsonl']
bn 234
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/hi
hi
['train.jsonl', 'test.jsonl', 'val.jsonl']
hi 207
/Users/rahulmehta/Desktop/Mult

In [51]:
#val_dataset = '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/' + lang + '_merged_test.jsonl

In [ ]:
# ent_dict = defaultdict()
# #fact_dict_all = defaultdict()
# cnts_dict = {}
# languages = []

# for dataset in all_datasets:
#     lang = dataset.split('/')[-1][:-6].split('_')[0]
#     print(lang)
    
#     e_dict = {}
    
#     cnts = []
    
#     with open(dataset) as f:
        
        
#         unique_facts = []

#         for line in f:
#             c = 0
#             data = json.loads(line)
#             f = []
#             for l in data['facts_list']:
#                 #print('sentence')
#                 f2 = []
#                 for fact in l:
#                     f2.append(fact[0])  
#                     # Generate facts 
#                     unique_facts.append(fact[0])  
#                 f.append(f2)
#                 #print(f)
#                 c+=1
            
#             cnts.append(c)
    
#             e_dict[data['entity_name']] = f
        
#     languages.append(lang)
#     #fact_dict_all[lang] = set(unique_facts)
#     print(lang,len(fact_dict_all[lang]))
#     ent_dict[lang] = e_dict
#     cnts_dict[lang] = cnts

In [72]:
cnts_dict_ent = defaultdict()
for i,lang in enumerate(cnts_dict):
    num_sent = pd.Series(cnts_dict[lang]).sum()
    #print(lang,num_sent)
    cnts_dict_ent[lang]=num_sent
print(cnts_dict_ent)

defaultdict(None, {'mr': 1432, 'gu': 415, 'te': 1411, 'pa': 2304, 'ml': 3607, 'kn': 1008, 'bn': 7550, 'hi': 3275, 'as': 485, 'en': 10216, 'or': 982, 'ta': 3731})


In [73]:
pd.Series(cnts_dict['te']).value_counts()

2     76
1     72
3     62
4     48
5     37
6     28
7     21
8     15
9     11
10     9
dtype: int64

##### Clustering on a dataset

In [29]:
# fact_dict = defaultdict()
# factsetdict = defaultdict()

# for lang in languages:
#     print(lang)
#     fdict = {i:f for i,f in enumerate(fact_dict_all[lang])}
    
#     factsetdict[lang] = set(fact_dict_all[lang])
    
#     fact_dict[lang] = fdict
#     #print(fact_dict) 

In [65]:
#fact_dict['en']

In [64]:
#fact_dict['en']

In [373]:
#fact_dict.values()

#### Label Matching on clustering

In [73]:
#fact_dict['en']

#### New logic for cluster matching

In [167]:
#if fact_dict['en'].value

In [74]:
languages

['mr', 'gu', 'te', 'pa', 'ml', 'kn', 'bn', 'hi', 'as', 'en', 'or', 'ta']

In [76]:
!pip install munkres

  Using cached munkres-1.1.4-py2.py3-none-any.whl (7.0 kB)


In [78]:
from labelmapping import label_matching,make_cost_matrix,translate_clustering,accuracy

In [101]:
#ent_dict 
# for e,c in zip(ent_dict.keys(),cnts):
#     print(e,c)
from collections import Counter
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/affinity_matrices/"

entpred_dict_all = defaultdict()

cnt =0
nothing=0
macro_pr = []
macro_rec = []
macro_pr_old = []
macro_rec_old = []
facts_len = 0
weighted_acc =[]
missed = []
scores_dict = defaultdict()

for lang in languages:
    #if lang == 'ml':

    scores = defaultdict()
    missed = 0

    print(lang)
    affinity_matrix = np.load(root + lang + '_affinity_matrix.npy')
    print("loaded affinity Matrix")
    print(affinity_matrix.shape)

    entpred_dict  = defaultdict()

    for i,v in enumerate(ent_dict[lang].keys()):

        cnt+=1
#             if cnt>10:
#                 break
#        print(i)
        #print(v,ent_dict[v])
        e = ent_dict[lang][v]
        c = len(ent_dict[lang][v])

#         print("v",v)
#         print("e",e)
#         print("c",c)

        #print(flat_list)


        # Get uniques facts 

        flat_list = [item for sublist in e for item in sublist]

        flat_set = set(flat_list)
        #print(flat_set)
        flat_ids = []

        for item in flat_set:
            for i,(key,value) in enumerate(fact_dict[lang].items()):
                if value==item:
                    #print(value)
                    flat_ids.append(key)
        #print(flat_ids)


        # Get ids
        flat_list_cnts = dict(Counter(flat_list))
        #print(flat_list_cnts)

        ent = e
        for i,value in enumerate(ent):
            for j,subitem in enumerate(value):
                #print(ent[i][j])
                for lookupitem in flat_list_cnts.keys():
                    #print(lookupitem)
                    if subitem == lookupitem and flat_list_cnts[lookupitem]!=1:    
                        ent[i][j] = tuple([ent[i][j],lookupitem + '_' + str(flat_list_cnts[lookupitem])])
                        flat_list_cnts[lookupitem] -= 1
                        #print(flat_list_cnts)
                        #print(e)
                        continue
                #print(type(ent[i][j]))
                if not isinstance(ent[i][j], tuple):
                    #print(True)
                    ent[i][j] = tuple([ent[i][j],ent[i][j]])


        #print("Ent",ent)
        flat_ids = []
        sent_id=0
        act_list = []
        flag=0
        new_fact_id = 200000
        for sent in ent:
            for item in sent:
                #print("item[0]",item[0])
                for i,(key,value) in enumerate(fact_dict[lang].items()):
                    if value==item[0]:
                        flag==1
                        flat_ids.append(key)
                        act_list.append(sent_id)
                        break
#                     flat_ids.append(new_fact_id)
#                     new_fact_id +=1
#                break

            sent_id +=1

#        print("Actual Clusters",act_list)
#         print(act_list)

        ent_perm_list = list(permutations(flat_ids,2))


        ent_aff_matrix = np.zeros((len(flat_ids),len(flat_ids)))
        #print(ent_aff_matrix.shape)


        #print(ent_perm_list)
        for p in ent_perm_list:
            ent_aff_matrix[flat_ids.index(p[0]),flat_ids.index(p[1])] = affinity_matrix[p[0],p[1]]

        #print(ent_aff_matrix)


        # clusters
        try:
            #ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
            #print("Clusters",ent_clusters)
            ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
            #print("predicted Clusters",ent_clusters)
        except Exception:
            #nothing +=1
            #print(nothing)
            #print("No clusters")
            #print(ent_aff_matrix.shape,c)
            continue


        # LABEL MAPPING
        if (pd.Series(act_list).nunique() == pd.Series(ent_clusters).nunique()):
            new_clusters,cm,old_acc,old_pr,old_re,new_cm,new_acc,new_pr,new_re  = label_matching(act_list,ent_clusters)
        else:
            #print("Missed entity")
            missed +=1
            print("missed")

        #print("New Clusters",new_clusters)

        macro_pr.append(new_pr)
        macro_rec.append(new_re)


        # Weighted accuracy
        #print(new_acc)
        weighted_acc.append(new_acc * len(flat_ids))
        facts_len+= len(flat_ids)

        macro_pr_old.append(old_pr)
        macro_rec_old.append(old_re)


        entpred_dict[v] = new_clusters

    scores['missed_ent'] = missed
    macro_pr_total = pd.Series(macro_pr).sum()/cnt
    macro_rec_total = pd.Series(macro_rec).sum()/cnt
    weighted_acc_total = pd.Series(weighted_acc).sum()/facts_len
    macro_pr_old_total = pd.Series(macro_pr_old).sum()/cnt
    macro_rec_old_total = pd.Series(macro_rec_old).sum()/cnt

    scores['macro_pr'] = round(macro_pr_total*100,2)
    scores['macro_rec'] = round(macro_rec_total*100,2)
    scores['weighted_acc'] = round(weighted_acc_total*100,2)
    scores['macro_pr_old'] = round(macro_pr_old_total*100,2)
    scores['macro_rec_old'] = round(macro_rec_old_total*100,2)
    print(scores)
    scores_dict[lang] = scores

mr
loaded affinity Matrix
(172, 172)
missed
missed
missed
defaultdict(None, {'missed_ent': 3, 'macro_pr': 91.38, 'macro_rec': 89.55, 'weighted_acc': 83.16, 'macro_pr_old': 76.87, 'macro_rec_old': 75.97})
gu
loaded affinity Matrix
(122, 122)
missed
missed
defaultdict(None, {'missed_ent': 2, 'macro_pr': 90.03, 'macro_rec': 88.53, 'weighted_acc': 80.86, 'macro_pr_old': 72.97, 'macro_rec_old': 72.14})
te
loaded affinity Matrix
(151, 151)
defaultdict(None, {'missed_ent': 0, 'macro_pr': 88.06, 'macro_rec': 86.68, 'weighted_acc': 77.25, 'macro_pr_old': 64.7, 'macro_rec_old': 64.01})
pa
loaded affinity Matrix
(201, 201)
missed
missed
defaultdict(None, {'missed_ent': 2, 'macro_pr': 85.38, 'macro_rec': 83.98, 'weighted_acc': 73.84, 'macro_pr_old': 60.78, 'macro_rec_old': 60.19})
ml
loaded affinity Matrix
(223, 223)
missed
defaultdict(None, {'missed_ent': 1, 'macro_pr': 84.57, 'macro_rec': 83.29, 'weighted_acc': 73.07, 'macro_pr_old': 57.53, 'macro_rec_old': 57.04})
kn
loaded affinity Matrix
(150

In [102]:
scores_dict

defaultdict(None,
            {'mr': defaultdict(None,
                         {'missed_ent': 3,
                          'macro_pr': 91.38,
                          'macro_rec': 89.55,
                          'weighted_acc': 83.16,
                          'macro_pr_old': 76.87,
                          'macro_rec_old': 75.97}),
             'gu': defaultdict(None,
                         {'missed_ent': 2,
                          'macro_pr': 90.03,
                          'macro_rec': 88.53,
                          'weighted_acc': 80.86,
                          'macro_pr_old': 72.97,
                          'macro_rec_old': 72.14}),
             'te': defaultdict(None,
                         {'missed_ent': 0,
                          'macro_pr': 88.06,
                          'macro_rec': 86.68,
                          'weighted_acc': 77.25,
                          'macro_pr_old': 64.7,
                          'macro_rec_old': 64.01}),
             'pa': d

In [104]:
cols  =['language','total_ent','total_sent_all','macro_pr','macro_rec','weighted_acc','macro_pr_old','macro_rec_old']
df = pd.DataFrame(columns=cols)

for i,lang in enumerate(scores_dict):
    #print(lang)
    df2 = [{'language': lang, 'total_ent':len(ent_dict[lang]),
            'total_sent_all':cnts_dict_ent[lang],
#           'total_sent':scores_dict[lang]['total_sent'],
#            'total_sent_cov':round(scores_dict[lang]['total_sent']/cnts_dict_ent[lang],2),
        'missed_ent':scores_dict[lang]['missed_ent'],
       'macro_pr':scores_dict[lang]['macro_pr'],
        'macro_rec':scores_dict[lang]['macro_rec'],
        'weighted_acc':scores_dict[lang]['weighted_acc']
        ,'macro_pr_old':scores_dict[lang]['macro_pr_old']
        ,'macro_rec_old':scores_dict[lang]['macro_rec_old']}]
    #print(df2)
    df2= pd.DataFrame.from_dict(df2,orient='columns')
    #print(df2)
    
    df = pd.concat([df,df2], ignore_index=True)
    #df.append(df2)
    
print(df)    

df.to_csv("../results/clustering-precisionrecall-alllang-24Jan2023.csv",index=False)

   language total_ent total_sent_all  macro_pr  macro_rec  weighted_acc  \
0        mr       722           1432     91.38      89.55         83.16   
1        gu       121            415     90.03      88.53         80.86   
2        te       379           1411     88.06      86.68         77.25   
3        pa       731           2304     85.38      83.98         73.84   
4        ml      1117           3607     84.57      83.29         73.07   
5        kn       273           1008     84.37      83.14         72.92   
6        bn      1978           7550     82.75      80.86         71.15   
7        hi      1237           3275     83.59      81.79         71.81   
8        as       111            485     83.53      81.75         71.76   
9        en      4282          10216     84.96      83.17         72.96   
10       or       236            982     84.85      83.06         72.85   
11       ta      1340           3731     85.13      83.48         73.06   

    macro_pr_old  macro_

In [179]:
# macro_pr = pd.Series(macro_pr).sum()/cnt
# macro_rec = pd.Series(macro_rec).sum()/cnt
# weighted_acc = pd.Series(weighted_acc).sum()/facts_len
# macro_pr_old = pd.Series(macro_pr_old).sum()/cnt
# macro_rec_old = pd.Series(macro_rec_old).sum()/cnt
# print("Macro Precision",(macro_pr,macro_pr_old))
# print("Macro Recall",(macro_rec,macro_rec_old))
# print("Weighted_acc",weighted_acc)

Macro Precision (0.9090909090909091, 0.37878787878787873)
Macro Recall (0.9090909090909091, 0.37878787878787873)
Weighted_acc 1.0


##### Store validation cluster predictions

In [207]:
#entpred_dict_all['te']

In [82]:
entpred_dict_all.keys()

dict_keys([])

In [105]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/cluster_predictions/"
with open(root + 'cluster_predictions_all.pkl', 'wb') as f:
    pickle.dump(entpred_dict_all, f)

##### Accuracy on validation set

In [213]:
#len(entpred_dict_all['en'])

In [ ]:
scores_dict

In [106]:
cols  =['language','total_ent','total_sent_all','macro_pr','macro_rec','weighted_acc','macro_pr_old','macro_rec_old']
df = pd.DataFrame(columns=cols)

for i,lang in enumerate(scores_dict):
    #print(lang)
    df2 = [{'language': lang, 'total_ent':len(ent_dict[lang]),
            'total_sent_all':cnts_dict_ent[lang],
#           'total_sent':scores_dict[lang]['total_sent'],
#            'total_sent_cov':round(scores_dict[lang]['total_sent']/cnts_dict_ent[lang],2),
        'missed_ent':scores_dict[lang]['missed_ent'],
       'macro_pr':scores_dict[lang]['macro_pr'],
        'macro_rec':scores_dict[lang]['macro_rec'],
        'weighted_acc':scores_dict[lang]['weighted_acc']
        ,'macro_pr_old':scores_dict[lang]['macro_pr_old']
        ,'macro_rec_old':scores_dict[lang]['macro_rec_old']}]
    #print(df2)
    df2= pd.DataFrame.from_dict(df2,orient='columns')
    #print(df2)
    
    df = pd.concat([df,df2], ignore_index=True)
    #df.append(df2)
    
print(df)    

df.to_csv("../results/clustering-precisionrecall-alllang-24Jan2023.csv",index=False)

   language total_ent total_sent_all  macro_pr  macro_rec  weighted_acc  \
0        mr       722           1432     91.38      89.55         83.16   
1        gu       121            415     90.03      88.53         80.86   
2        te       379           1411     88.06      86.68         77.25   
3        pa       731           2304     85.38      83.98         73.84   
4        ml      1117           3607     84.57      83.29         73.07   
5        kn       273           1008     84.37      83.14         72.92   
6        bn      1978           7550     82.75      80.86         71.15   
7        hi      1237           3275     83.59      81.79         71.81   
8        as       111            485     83.53      81.75         71.76   
9        en      4282          10216     84.96      83.17         72.96   
10       or       236            982     84.85      83.06         72.85   
11       ta      1340           3731     85.13      83.48         73.06   

    macro_pr_old  macro_

In [107]:
df 

,language,total_ent,total_sent_all,macro_pr,macro_rec,weighted_acc,macro_pr_old,macro_rec_old,missed_ent
0,mr,722,1432,91.38,89.55,83.16,76.87,75.97,3.0
1,gu,121,415,90.03,88.53,80.86,72.97,72.14,2.0
2,te,379,1411,88.06,86.68,77.25,64.70,64.01,0.0
3,pa,731,2304,85.38,83.98,73.84,60.78,60.19,2.0
4,ml,1117,3607,84.57,83.29,73.07,57.53,57.04,1.0
5,kn,273,1008,84.37,83.14,72.92,56.42,55.96,1.0
6,bn,1978,7550,82.75,80.86,71.15,52.06,51.70,1.0
7,hi,1237,3275,83.59,81.79,71.81,53.95,53.63,0.0
8,as,111,485,83.53,81.75,71.76,53.71,53.40,2.0
9,en,4282,10216,84.96,83.17,72.96,59.01,58.60,1.0


In [108]:
df.to_csv("../results/clustering-scores-alllang-24Jan2022.csv",index=False)